In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [97]:
main_df = pd.read_json('../a-fediverse-stats.txt', orient='records', lines=True)
main_df

,endpoint,checked_target,result
0,https://ff.mizunashi.work/,ff.mizunashi.work,"{'type': 'ok', 'node_info': {'resource_type': ..."
1,https://atsuchan.page/,atsuchan.page,"{'type': 'ok', 'node_info': {'resource_type': ..."
2,https://m.huling.org/,m.huling.org,"{'type': 'ok', 'node_info': {'resource_type': ..."
3,https://types.pl/,types.pl,"{'type': 'ok', 'node_info': {'resource_type': ..."
4,https://lightcrystal.fedicity.net/,lightcrystal.fedicity.net,"{'type': 'ok', 'node_info': {'resource_type': ..."
...,...,...,...
37824,https://crazystreams.org/,crazystreams.org,"{'type': 'fail', 'resource_status': 'not-suppo..."
37825,https://mastodon.mwegner.com/,mastodon.mwegner.com,"{'type': 'fail', 'resource_status': 'unknown',..."
37826,https://pixelfed.lundgrensjostrom.com/,pixelfed.lundgrensjostrom.com,"{'type': 'ok', 'node_info': {'resource_type': ..."
37827,https://social.touha.me/,social.touha.me,"{'type': 'ok', 'node_info': {'resource_type': ..."


In [98]:
result_df = pd.json_normalize(main_df['result'])
result_df

,type,peers_count,node_info.resource_type,node_info.resource_url,node_info.node_name,node_info.protocols,node_info.services_inbound,node_info.services_outbound,node_info.software_name,node_info.software_version,...,node_info.maintainer_name,node_info.open_registrations,node_info.email_required_for_signup,node_info.enable_email,node_info.enable_hcaptcha,node_info.enable_recaptcha,node_info.langs,node_info.max_note_text_length,resource_status,detail
0,ok,1423.0,http://nodeinfo.diaspora.software/ns/schema/2.1,https://ff.mizunashi.work/nodeinfo/2.1,Mizunashi Social,[activitypub],[],"[atom1.0, rss2.0]",firefish,1.0.4-beta31,...,Mizunashi Mana,False,False,True,False,True,[],3000.0,NaN,NaN
1,ok,NaN,http://nodeinfo.diaspora.software/ns/schema/2.0,https://atsuchan.page/nodeinfo/2.0,NaN,[activitypub],NaN,NaN,mastodon,3.4.10+glitch+atsu1125,...,maintenance,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ok,1020.0,http://nodeinfo.diaspora.software/ns/schema/2.1,https://m.huling.org/nodeinfo/2.1,Huling 鯖,[activitypub],[],"[atom1.0, rss2.0]",misskey,2023.10.0-poskey-4ba6f45,...,xiupos,False,False,False,False,False,[],3000.0,NaN,NaN
3,ok,NaN,http://nodeinfo.diaspora.software/ns/schema/2.0,https://types.pl/nodeinfo/2.0,NaN,[activitypub],[],[],mastodon,4.2.0+types,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ok,1501.0,http://nodeinfo.diaspora.software/ns/schema/2.0,https://lightcrystal.fedicity.net/nodeinfo/2.0,NaN,[activitypub],[],[],mastodon,4.2.1,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37824,fail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-supported,Failed to fetch https://crazystreams.org/.well...
37825,fail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,Failed to fetch https://mastodon.mwegner.com/....
37826,ok,NaN,http://nodeinfo.diaspora.software/ns/schema/2.0,https://pixelfed.lundgrensjostrom.com/api/node...,Pixelfed@LundgrenSjostrom.com,[activitypub],[],[],pixelfed,0.11.9,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37827,ok,19614.0,http://nodeinfo.diaspora.software/ns/schema/2.0,https://social.touha.me/nodeinfo/2.0,NaN,[activitypub],[],[],mastodon,4.2.1,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
df = pd.DataFrame(result_df, columns=['type'])
df.loc[:, 'total'] = 1
df = df.pivot_table(index=['type'], aggfunc='count', values=['total'], margins=True)
df.loc[:, '%'] = (df['total'] / df.loc['All', 'total']) * 100
df

,total,%
type,,
fail,22848,60.398107
ok,14981,39.601893
All,37829,100.000000


In [115]:
def detect_detail_kind(detail):
    if type(detail) is not str:
        return detail
    elif re.match(r'.* getaddrinfo ENOTFOUND ', detail):
        return 'Addr not found'
    elif re.match(r'.* timeout of [0-9]+ms exceeded', detail):
        return 'Timeout'
    elif re.match(r'.*(,| )(connect ENETUNREACH|connect EHOSTUNREACH)', detail):
        return 'Connection unreachable'
    elif re.match(r'.* (self-signed certificate|Hostname/IP does not match certificate\'s altnames|certificate has expired|unable to verify the first certificate|unable to get local issuer certificate)', detail):
        return 'TLS certificate is invalid'
    elif re.match(r'.* invalid status=(401|402|403|406|418|420|423|429|451|5[0-9]+|999)', detail):
        return 'Invalid HTTP status'
    elif re.match(r'.*(,| )connect ECONNREFUSED', detail):
        return 'Connection refused'
    elif re.match(r'.*:SSL routines:.*(:SSL alert number [0-9]+|:wrong version number|:unsupported protocol)', detail):
        return 'Not supported TLS'
    elif re.match(r'.* getaddrinfo EAI_AGAIN', detail):
        return 'Failed to resolve addr'
    elif re.match(r'.* Client network socket disconnected before secure TLS connection was established', detail):
        return 'Failed to handshake TLS sessions'
    elif re.match(r'.* (socket hang up|read ECONNRESET)', detail):
        return 'Connection cancelled'
    elif re.match(r'.*(,| )connect EINVAL', detail):
        return 'Connection invalid'
    elif re.match(r'.* Maximum number of redirects exceeded', detail):
        return 'Too many redirections'
    elif re.match(r'.* Parse Error', detail):
        return 'Not supported HTTP'
    else:
        return '_'

df = pd.DataFrame(result_df, columns=['type', 'resource_status', 'detail'])
df.loc[:, 'detail_kind'] = df['detail'].apply(detect_detail_kind)  
df.loc[:, 'total'] = 1
df = df.pivot_table(index=['type', 'resource_status', 'detail_kind'], values=['total'], aggfunc='count').sort_values(by=['total'], ascending=False)
df.loc['All', 'total'] = df['total'].sum()
df.loc[:, '%'] = (df['total'] / df.loc['All', 'total'].iloc[0]) * 100
#df[(df['resource_status']=='unknown')&(df['detail_kind']=='_')]
df

/tmp/ipykernel_14771/3466141144.py:38: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[:, '%'] = (df['total'] / df.loc['All', 'total'].iloc[0]) * 100


total           %
type resource_status detail_kind                                          
fail unknown         Addr not found                     7602.0   33.272059
     not-supported   _                                  3834.0   16.780462
     unknown         Timeout                            2980.0   13.042717
                     Connection unreachable             2889.0   12.644433
                     TLS certificate is invalid         2678.0   11.720938
                     Invalid HTTP status                1338.0    5.856092
                     Connection refused                  518.0    2.267157
                     Not supported TLS                   436.0    1.908263
     gone            _                                   286.0    1.251751
     unknown         Failed to resolve addr              121.0    0.529587
                     Failed to handshake TLS sessions     99.0    0.433298
                     Connection cancelled                 57.0    0.249475
                     Connection invalid                    7.0    0.030637
                     Too many redirections                 2.0    0.008754
                     Not supported HTTP                    1.0    0.004377
All                                                    22848.0  100.000000